In [1]:
print("hi")

hi


In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymongo as mongo
import os

In [91]:
Imgpath = "D:\\Download\\archive\\SubImage"
dataframe_path = "D:\\Download\\archive\\captions.txt"

In [92]:
list_images = os.listdir(Imgpath)
len(list_images)

825

In [93]:
df = pd.read_csv(dataframe_path)
df.head(10)

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
5,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
6,1001773457_577c3a7d70.jpg,A black dog and a tri-colored dog playing with...
7,1001773457_577c3a7d70.jpg,A black dog and a white dog with brown spots a...
8,1001773457_577c3a7d70.jpg,Two dogs of different breeds looking at each o...
9,1001773457_577c3a7d70.jpg,Two dogs on pavement moving toward each other .


In [7]:
list_images[:10]

['101654506_8eb26cfb60.jpg',
 '101669240_b2d3e7f17b.jpg',
 '102351840_323e3de834.jpg',
 '102455176_5f8ead62d5.jpg',
 '103106960_e8a41d64f8.jpg',
 '103195344_5d2dc613a3.jpg',
 '103205630_682ca7285b.jpg',
 '104136873_5b5d41be75.jpg',
 '105342180_4d4a40b47f.jpg',
 '106490881_5a2dd9b7bd.jpg']

In [94]:
df2 = df[df['image'].isin(list_images)]

In [95]:
df2.shape

(4125, 2)

In [96]:
df2.groupby("image").count().shape

(825, 1)

In [97]:
len(list_images)

825

In [50]:
df2[df2['image']=="1000268201_693b08cb0e.jpg"]

,image,caption


In [66]:
df2.groupby("image")["caption"].apply(list).to_dict()

{'101654506_8eb26cfb60.jpg': ['A brown and white dog is running through the snow .',
  'A dog is running in the snow',
  'A dog running through snow .',
  'a white and brown dog is running through a snow covered field .',
  'The white and brown dog is running over the surface of the snow .'],
 '101669240_b2d3e7f17b.jpg': ['A man in a hat is displaying pictures next to a skier in a blue hat .',
  'A man skis past another man displaying paintings in the snow .',
  'A person wearing skis looking at framed pictures set up in the snow .',
  'A skier looks at framed pictures in the snow next to trees .',
  'Man on skis looking at artwork for sale in the snow'],
 '102351840_323e3de834.jpg': ['A man drilling a hole in the ice .',
  'A man is drilling through the frozen ice of a pond .',
  'A person in the snow drilling a hole in the ice .',
  'A person standing on a frozen lake .',
  'Two men are ice fishing .'],
 '102455176_5f8ead62d5.jpg': ['A man uses ice picks and crampons to scale ice .',

In [17]:
df2.to_csv("D:\\Download\\archive\\Finalcaption.csv")

In [33]:
from dotenv import load_dotenv

load_dotenv()

True

## MongoDB Upload and Download Functions

In [98]:
import os
import base64
import pandas as pd
from pymongo import MongoClient

def upload_images_and_captions_to_mongodb(image_folder, csv_file, mongo_uri, db_name, collection_name):
    counter = 0
    # Connect to MongoDB
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]

    # Load the CSV file
    df = pd.read_csv(csv_file)

    # Group the captions by image name
    image_captions = df.groupby('image')['caption'].apply(list).to_dict()
    print("dataframe created")

    # Loop through the image files
    for image_name, captions in image_captions.items():
        image_path = os.path.join(image_folder, image_name)

        # Check if the image file exists
        if os.path.exists(image_path):
            # Read and encode the image file
            with open(image_path, 'rb') as image_file:
                encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
            
            # Create a document to insert into MongoDB
            document = {
                'image_name': image_name,
                'captions': captions,
                'image_data': encoded_image
            }

            # Insert the document into MongoDB
            collection.insert_one(document)
        else:
            print(f'Image {image_name} not found in the folder.')
            counter += 1

    print('Upload complete! ',counter)

def download_images_and_captions_from_mongodb(mongo_uri, db_name, collection_name, download_folder, output_csv):
    # Connect to MongoDB
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]

    # Find all documents in the collection
    documents = collection.find()

    # Prepare a list to store the caption data for CSV
    csv_data = []

    os.makedirs(download_folder, exist_ok=True)
    # Loop through the documents and save the images
    for doc in documents:
        image_name = doc['image_name']
        encoded_image = doc['image_data']
        captions = doc['captions']
        image_path = os.path.join(download_folder, image_name)

        # Decode and save the image file
        with open(image_path, 'wb') as image_file:
            image_file.write(base64.b64decode(encoded_image))
        
        # Add the captions to the CSV data list
        for caption in captions:
            csv_data.append({'image': image_name, 'caption': caption})

    # Save the CSV data to a file
    df = pd.DataFrame(csv_data)
    df.to_csv(output_csv, index=False)

    print('Download complete!')


In [45]:
os.path.getsize(os.path.join(image_folder, "101654506_8eb26cfb60.jpg"))

101800

In [99]:
image_folder = "D:\\Download\\archive\\SubImage"
csv_file = "D:\\Download\\archive\\captions.txt"

mongo_uri = os.environ.get('MONGO_URI')
#print(mongo_uri)
db_name = 'IMAGECAPTION'
collection_name = 'image_data'

upload_images_and_captions_to_mongodb(image_folder, csv_file, mongo_uri, db_name, collection_name)


dataframe created
Image 1000268201_693b08cb0e.jpg not found in the folder.
Image 1001773457_577c3a7d70.jpg not found in the folder.
Image 1002674143_1b742ab4b8.jpg not found in the folder.
Image 1003163366_44323f5815.jpg not found in the folder.
Image 1007129816_e794419615.jpg not found in the folder.
Image 1007320043_627395c3d8.jpg not found in the folder.
Image 1009434119_febe49276a.jpg not found in the folder.
Image 1012212859_01547e3f17.jpg not found in the folder.
Image 1015118661_980735411b.jpg not found in the folder.
Image 1015584366_dfcec3c85a.jpg not found in the folder.
Image 1016887272_03199f49c4.jpg not found in the folder.
Image 1019077836_6fc9b15408.jpg not found in the folder.
Image 1019604187_d087bf9a5f.jpg not found in the folder.
Image 1020651753_06077ec457.jpg not found in the folder.
Image 1022454332_6af2c1449a.jpg not found in the folder.
Image 1022454428_b6b660a67b.jpg not found in the folder.
Image 1022975728_75515238d8.jpg not found in the folder.
Image 1024138

In [85]:
# Download Images and Captions
download_folder = 'C:\\Project\\Automatic-Image-Captioning\\downloads\\Images'
output_csv = 'C:\\Project\\Automatic-Image-Captioning\\downloads\\captions.csv'

download_images_and_captions_from_mongodb(mongo_uri, db_name, collection_name, download_folder, output_csv)


Download complete!


In [86]:
df3 = pd.read_csv(output_csv)
df3.head()

,image,caption
0,101654506_8eb26cfb60.jpg,A brown and white dog is running through the s...
1,101654506_8eb26cfb60.jpg,A dog is running in the snow
2,101654506_8eb26cfb60.jpg,A dog running through snow .
3,101654506_8eb26cfb60.jpg,a white and brown dog is running through a sno...
4,101654506_8eb26cfb60.jpg,The white and brown dog is running over the su...


In [72]:
list_images = os.listdir(download_folder)
len(list_images)

222

# data validation steps

In [88]:
bad_name = []
good_name = []
zero_size_name = []
csv_image_name = set(df3["image"])
for img in list_images:
    name, ext = img.split('.')
    img_path = os.path.join(download_folder, img)
    if ext != "jpg" or img not in csv_image_name:
        print(f"{img} has invalid extension. Only jpg files are allowed.")
        bad_name.append(img)
        #os.remove(img_path)
    elif os.path.getsize(img_path) == 0:
        print(f"{img} has zero size.")
        zero_size_name.append(img)
        #os.remove(img_path)
    else:
        good_name.append(img)

print(f"bad name length : {len(bad_name)}")
print(f"good name length : {len(good_name)}")
print(f"zero size length : {len(zero_size_name)}")


bad name length : 0
good name length : 222
zero size length : 0


In [90]:
os.path.getsize('C:\\Project\\Automatic-Image-Captioning\\downloads\\Images\\101654506_8eb26cfb60.jpg')

101800

In [83]:
df3.shape

(1110, 2)